In [1]:
import pandas as pd
import numpy as np
import os
from Functions import matching_check

## Loading files and Pre processing

In [2]:
# read files
FA = pd.read_csv('betsi_origin/FA.csv', sep=';' ,encoding='iso-8859-1', decimal=',')
FB_Klimat = pd.read_csv('betsi_origin/FB_Klimat.csv',sep=';' ,encoding='iso-8859-1', decimal=',').drop(columns=['Hustyp'])
year = pd.read_csv('betsi_origin/Byggår.csv', encoding='iso-8859-1')
FB49_YTTERVAGG = pd.read_csv('betsi_origin/FB49 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')
FB52_YTTERVAGG = pd.read_csv('betsi_origin/FB52 Ytterv„gg.csv', sep=';', encoding='iso-8859-1', decimal=',')

In [ ]:
# concat several features
F = pd.concat([FA, FB_Klimat],axis=1, join='inner')

# Delete unrealistic values
F = F[(F['FB52 U-medelvärde ytterväggar']>=0.1) & (F['FB87-97 U-medelvärde fönster']>0)]
F.loc[:, 'FXByggår'] = year['FXByggÃ¥r']

# select muilti-family buildings
F = F[F['Hustyp']=='F']

# Insert classification column
F.insert(loc=0, column='Year', value=0)
F.insert(loc=1, column='Classification', value=0)

In [ ]:
# create varant names for separation 
classification = []
for i in ['F_60', 'F_61_75', 'F_76_85', 'F_86_95', 'F_96_05']:
    for j in ['Lamellhus', 'Skivhus', 'Punkthus', 'Loftgångshus', 'Flerbostadsvilla', 'AnnanVilken', 'Radhus']:
        # print("{}_{} = {}[{}['FA11Byggnadstyp']=='{}']".format(i, j, i, i, j))
        classification.append('{}_{}'.format(i, j)) # lack of F_76_85_Sutteränghus(but after value check there's no)
        # print('{}_{}'.format(i, j))  

In [ ]:
# devide data into several decades
F_60 = F[F['FXByggår']<=1960]
F_61_75 = F[(F['FXByggår']>=1961) & (F['FXByggår'] <= 1975)]
F_76_85 = F[(F['FXByggår']>=1976) & (F['FXByggår'] <= 1985)]
F_86_95 = F[(F['FXByggår']>=1986) & (F['FXByggår'] <= 1995)]
F_96_05 = F[(F['FXByggår']>=1996) & (F['FXByggår'] <= 2005)]
check = pd.concat([F_60, F_61_75, F_76_85, F_86_95, F_96_05])
# pd.set_option('display.max_rows', None)
U_wall = []
U_win = []

In [ ]:
# for i in classification:
#     print("U_wall.append({}['FB52 U-medelvärde ytterväggar'].mean())".format(i))
#     print("U_win.append({}['FB87-97 U-medelvärde fönster'].mean())".format(i))


In [ ]:
F_60.loc[:, 'Year'] = 'F_60'
F_61_75.loc[:, 'Year'] = 'F_61_75'
F_76_85.loc[:, 'Year'] = 'F_76_85'
F_86_95.loc[:, 'Year'] = 'F_86_95'
F_96_05.loc[:, 'Year'] = 'F_96_05'
F = pd.concat([F_60, F_61_75, F_76_85, F_86_95, F_96_05])

In [ ]:
# F.insert(loc=1, column='Classification')
F.loc[:, 'Classification'] = F['Year'].values +'_'+F['FA11Byggnadstyp'].values
F

### Agregated by years

In [ ]:
F = F.T.drop_duplicates().T
F.loc[:, 'Classification'] = list(F['Year'])
F

### All for one

In [ ]:
F.loc[:, 'Classification'] = 'BESTI'

## Calculation of U-value Roof (incl. cold bridge)

In [ ]:
# Roof U-values
roof = F[['Year', 
          'Classification', 
          'Byggnad', 
          'F14ATempt', 
          'FA16AntalPlanOvanMark',
          'FB56 Area horisontell vindsbjälklag m2',
          'FB56 U-värde vindsbjälklag',
          'FB56 UA-värde horizontellt vindsbjälklag',
          'FB56 UA-värde horizontellt vindsbjälklag inkl köldbryggor',
          'FB58 Area stödbensväggar',
          'FB59 U-värde stödbensväggar',
          'FB58 UA-värde stödbensväggar',
          'FB58 UA-värde stödbensväggar inkl köldbryggor',
          'FB61 Area snedtak',
          'FB62 U-värde snedtak',
          'FB62 UA-värde snedtak',
          'FB62 UA-värde snedtak inkl köldbryggor']]

# drop wrong conditions
roof = roof.drop(roof[roof['FB56 Area horisontell vindsbjälklag m2'].notnull() & 
                  ((roof['FB56 U-värde vindsbjälklag'].isnull()) | 
                   ((roof['FB56 U-värde vindsbjälklag'] == 0)))].index)

roof = roof.drop(roof[roof['FB58 Area stödbensväggar'].notnull() & 
                  ((roof['FB59 U-värde stödbensväggar'].isnull()) | 
                   ((roof['FB59 U-värde stödbensväggar'] == 0)))].index)

roof = roof.drop(roof[roof['FB61 Area snedtak'].notnull() & 
                  ((roof['FB62 U-värde snedtak'].isnull()) | 
                   ((roof['FB62 U-värde snedtak'] == 0)))].index)

roof = roof.drop(roof[roof['FB61 Area snedtak'].isnull() & 
                 roof['FB56 Area horisontell vindsbjälklag m2'].isnull() &
                 roof['FB61 Area snedtak'].isnull()].index)

roof = roof.replace(np.nan, 0)

In [ ]:
# calculate average U-roof and Area of roof
Uroof = []
Aroof = []
for i in range(len(list(roof['Year']))):
    fv = 0
    fst = 0
    fsn = 0
    
    if roof['FB56 UA-värde horizontellt vindsbjälklag'].iloc[i] != 0:
        fv = roof['FB56 UA-värde horizontellt vindsbjälklag inkl köldbryggor'].iloc[i]/roof['FB56 UA-värde horizontellt vindsbjälklag'].iloc[i]
    if roof['FB58 UA-värde stödbensväggar'].iloc[i] != 0:
        fst = roof['FB58 UA-värde stödbensväggar inkl köldbryggor'].iloc[i]/roof['FB58 UA-värde stödbensväggar'].iloc[i]
    if roof['FB62 UA-värde snedtak'].iloc[i] != 0:
        fsn = roof['FB62 UA-värde snedtak inkl köldbryggor'].iloc[i]/roof['FB62 UA-värde snedtak'].iloc[i]
    
    Uaf = (
        ((roof['FB56 U-värde vindsbjälklag'].iloc[i] * fv * roof['FB56 Area horisontell vindsbjälklag m2'].iloc[i]) + 
        (roof['FB59 U-värde stödbensväggar'].iloc[i] * fst * roof['FB58 Area stödbensväggar'].iloc[i]) +
        (roof['FB62 U-värde snedtak'].iloc[i] * fsn * roof['FB61 Area snedtak'].iloc[i]))/
        (roof['FB56 Area horisontell vindsbjälklag m2'].iloc[i] + roof['FB58 Area stödbensväggar'].iloc[i] + roof['FB61 Area snedtak'].iloc[i])
    )
    
    Uroof.append(Uaf)
    Aroof.append(roof['FB56 Area horisontell vindsbjälklag m2'].iloc[i] + roof['FB58 Area stödbensväggar'].iloc[i] + roof['FB61 Area snedtak'].iloc[i])
    
roof['U_roof'] = Uroof
roof['A_roof'] = Aroof

In [ ]:
# delete unrealistic vlaues of roof
roof = roof[(roof['U_roof']>=0.1) &
             (roof['U_roof']<=2)]

In [ ]:
# transfer data to archetypes
Arche_roof = roof.groupby('Classification').agg({'U_roof':'mean',
                                    'A_roof':'mean'})

## Calculation of WWR

In [ ]:
F.loc[:, 'FB88 Summa fönsterarea N'].replace(0, np.nan)
F.loc[:, 'FB91 Summa fönsterarea S'].replace(0, np.nan)
F.loc[:, 'FB94 Summa fönsterarea V'].replace(0, np.nan)
F.loc[:, 'FB97 Summa fönsterarea O'].replace(0, np.nan)
F_wwr = F

In [ ]:
F_wwr = F_wwr.drop(F_wwr[(F_wwr['FB48 Ytterväggsarea N']==0)|
                        (F_wwr['FB48 Ytterväggsarea S']==0)|
                        (F_wwr['FB48 Ytterväggsarea V']==0)|
                        (F_wwr['FB48 Ytterväggsarea O']==0)].index)
F_wwr

In [ ]:
wwr_N = F_wwr['FB88 Summa fönsterarea N'] / F_wwr['FB48 Ytterväggsarea N']
wwr_S = F_wwr['FB91 Summa fönsterarea S'] / F_wwr['FB48 Ytterväggsarea S']
wwr_V = F_wwr['FB94 Summa fönsterarea V'] / F_wwr['FB48 Ytterväggsarea V']
wwr_O = F_wwr['FB97 Summa fönsterarea O'] / F_wwr['FB48 Ytterväggsarea O']
WWR = F_wwr[['Year', 'Classification', 'Byggnad']]
WWR.loc[:, 'WWR_N'] = list(wwr_N)
WWR.loc[:, 'WWR_S'] = list(wwr_S)
WWR.loc[:, 'WWR_V'] = list(wwr_V)
WWR.loc[:, 'WWR_O'] = list(wwr_O)
Arche_WWR = WWR.groupby('Classification').agg({'WWR_N' : 'mean', 
                                 'WWR_S' : 'mean', 
                                 'WWR_V' : 'mean', 
                                 'WWR_O' : 'mean'})

## Calculation of U-value window

In [ ]:
F_window = F.drop(F[F['FB87-97 U-medelvärde fönster'] == 0].index)
Arche_window = F_window.groupby('Classification').agg({'FB87-97 U-medelvärde fönster' : 'mean'})
Arche_window = Arche_window.rename(columns={'FB87-97 U-medelvärde fönster':'U_widnows'})

## Calculation of U-exterior wall

In [ ]:
F_wall = F
Arche_wall = F_wall.groupby('Classification').agg({'FB52 U-medelvärde ytterväggar' : 'mean'})
Arche_wall = Arche_wall.rename(columns={'FB52 U-medelvärde ytterväggar':'U-wall'})

## Calculation of U-ground

In [ ]:
# Priority: U crawl space > U basement
F_ground1 = F[['FB33 U-värde platta på mark/källargolv', 
              'FB41 U-värde bjälklag ovan ouppvärmd källare',
              'FB30 U-värde bjälklag ovan krypgrund',
               'Classification']]

F_ground1 = F_ground1.drop(F_ground1[(F_ground1.iloc[:,0].isnull()) & 
                        (F_ground1.iloc[:,1].isnull()) &
                        (F_ground1.iloc[:,2].isnull())].index)
    
F_ground = pd.DataFrame(F_ground1[['FB33 U-värde platta på mark/källargolv', 'Classification']])
for i in range(len(list(F_ground1['FB33 U-värde platta på mark/källargolv']))):
    if pd.notnull(F_ground1.iloc[i,1])==True:
        F_ground.iloc[i, 0] = F_ground1.iloc[i,1]
    if pd.notnull(F_ground1.iloc[i, 2])==True:
        F_ground.iloc[i, 0] = F_ground1.iloc[i,2]

Arche_ground = F_ground.groupby('Classification').agg({'FB33 U-värde platta på mark/källargolv' : 'mean'})
Arche_ground = Arche_ground.rename(columns={'FB33 U-värde platta på mark/källargolv':'U-ground'})

## Calculation of floor height

In [ ]:
F_floor_basement = F[['FA19Kallare', 'Classification']].drop(F[F['FA19Kallare'] > 3.5].index)
F_floor_ground = F[['FA19Bottenvaning', 'Classification']].drop(F[F['FA19Bottenvaning'] > 3.5].index)
F_floor_typical = F[['FA19Vaningsplan', 'Classification']].drop(F[F['FA19Vaningsplan'] > 3.5].index)
F_floor_actic = F[['FA19InreddVind', 'Classification']].drop(F[F['FA19InreddVind'] > 3.5].index)

# Nan value ingored by mean 
Arche_floor = pd.concat([F_floor_basement.groupby('Classification').agg({'FA19Kallare':'mean'}), 
                        F_floor_ground.groupby('Classification').agg({'FA19Bottenvaning':'mean'}),
                        F_floor_typical.groupby('Classification').agg({'FA19Vaningsplan':'mean'}),
                        F_floor_actic.groupby('Classification').agg({'FA19InreddVind':'mean'})], axis=1)

Arche_floor = Arche_floor.rename(columns=({'FA19Kallare':'Floor height_Basement', 
                            'FA19Bottenvaning':'Floor height_Ground', 
                            'FA19Vaningsplan':'Floor height_Typical',
                            'FA19InreddVind':'Floor height_Acttic'}))

Arche_floor.loc[:, 'Average floor height'] = Arche_floor.mean(axis=1)
Arche_floor

## Crawl space and Basement check

In [ ]:
if_crwal = F[['Classification', 'FB10 Krypgrund/bjälklag ovan krypgrund']]
if_basement = F[['Classification', 'FB40 Källare', 'FB41 Källare uppvärmd']]
# list(if_basement['FB40 Källare']).count('Ja')

# ratio of having crawl space
ratio_crawl = []
for i in range(len(list(if_crwal.groupby('Classification')))):
    ja = list(list(if_crwal.groupby('Classification'))[i][1]['FB10 Krypgrund/bjälklag ovan krypgrund']).count('Ja')
    r = ja/(len(list(if_crwal.groupby('Classification'))[i][1]))
    ratio_crawl.append(r)

ratio_base = []
# ratio of having crawl space
for i in range(len(list(if_basement.groupby('Classification')))):
    ja1 = list(list(if_basement.groupby('Classification'))[i][1]['FB40 Källare']).count('Ja')
    r1 = ja1/(len(list(if_basement.groupby('Classification'))[i][1]))
    ratio_base.append(r1)

In [ ]:
# Archetype information for crawl space
Arche_crawl = pd.DataFrame(if_crwal.groupby('Classification').agg({'FB10 Krypgrund/bjälklag ovan krypgrund':';'.join}).loc[:, 'FB10 Krypgrund/bjälklag ovan krypgrund'])
Arche_crawl.iloc[:, 0] = ratio_crawl
Arche_crawl = Arche_crawl.rename(columns={'FB10 Krypgrund/bjälklag ovan krypgrund': 'Ratio of Crawl Space'})

In [ ]:
# Archetype information for basement
Arche_basement = pd.DataFrame(if_crwal.groupby('Classification').agg({'FB10 Krypgrund/bjälklag ovan krypgrund':';'.join}).loc[:, 'FB10 Krypgrund/bjälklag ovan krypgrund'])
Arche_basement.iloc[:, 0] = ratio_base
Arche_basement.loc[:, 'Uwall_base_under ground'] = F.groupby('Classification').agg({'FB47 U-värde källarvägg under mark':'mean'})
Arche_basement.loc[:, 'Uwall_base_above ground'] = F.groupby('Classification').agg({'FB47 U-värde källarvägg ovan mark':'mean'})

Arche_basement = Arche_basement.rename(columns={'FB10 Krypgrund/bjälklag ovan krypgrund': 'Ratio of Basement'})

## Archetypes Combination

In [ ]:
Arche = []
Arche = pd.concat([Arche_wall, 
                   Arche_window, 
                   Arche_ground, 
                   Arche_roof, 
                   Arche_WWR, 
                   Arche_floor,
                   Arche_crawl,
                   Arche_basement],axis=1)
Arche

In [ ]:
Arche.to_excel('BESTI_all.xlsx', sheet_name='sheet1')

## (Previous codes)

In [ ]:
# # seperation from decades to building types

# F_60_Lamellhus = F_60[F_60['FA11Byggnadstyp']=='Lamellhus']
# F_60_Skivhus = F_60[F_60['FA11Byggnadstyp']=='Skivhus']
# F_60_Punkthus = F_60[F_60['FA11Byggnadstyp']=='Punkthus']
# F_60_Loftgångshus = F_60[F_60['FA11Byggnadstyp']=='Loftgångshus']
# F_60_Flerbostadsvilla = F_60[F_60['FA11Byggnadstyp']=='Flerbostadsvilla']
# F_60_AnnanVilken = F_60[F_60['FA11Byggnadstyp']=='AnnanVilken']
# F_60_Radhus = F_60[F_60['FA11Byggnadstyp']=='Radhus']
# F_61_75_Lamellhus = F_61_75[F_61_75['FA11Byggnadstyp']=='Lamellhus']
# F_61_75_Skivhus = F_61_75[F_61_75['FA11Byggnadstyp']=='Skivhus']
# F_61_75_Punkthus = F_61_75[F_61_75['FA11Byggnadstyp']=='Punkthus']
# F_61_75_Loftgångshus = F_61_75[F_61_75['FA11Byggnadstyp']=='Loftgångshus']
# F_61_75_Flerbostadsvilla = F_61_75[F_61_75['FA11Byggnadstyp']=='Flerbostadsvilla']
# F_61_75_AnnanVilken = F_61_75[F_61_75['FA11Byggnadstyp']=='AnnanVilken']
# F_61_75_Radhus = F_61_75[F_61_75['FA11Byggnadstyp']=='Radhus']
# F_76_85_Lamellhus = F_76_85[F_76_85['FA11Byggnadstyp']=='Lamellhus']
# F_76_85_Skivhus = F_76_85[F_76_85['FA11Byggnadstyp']=='Skivhus']
# F_76_85_Punkthus = F_76_85[F_76_85['FA11Byggnadstyp']=='Punkthus']
# F_76_85_Loftgångshus = F_76_85[F_76_85['FA11Byggnadstyp']=='Loftgångshus']
# F_76_85_Flerbostadsvilla = F_76_85[F_76_85['FA11Byggnadstyp']=='Flerbostadsvilla']
# F_76_85_AnnanVilken = F_76_85[F_76_85['FA11Byggnadstyp']=='AnnanVilken']
# F_76_85_Radhus = F_76_85[F_76_85['FA11Byggnadstyp']=='Radhus']
# F_76_85_Sutteränghus = F_76_85[F_76_85['FA11Byggnadstyp']=='Sutteränghus']
# F_86_95_Lamellhus = F_86_95[F_86_95['FA11Byggnadstyp']=='Lamellhus']
# F_86_95_Skivhus = F_86_95[F_86_95['FA11Byggnadstyp']=='Skivhus']
# F_86_95_Punkthus = F_86_95[F_86_95['FA11Byggnadstyp']=='Punkthus']
# F_86_95_Loftgångshus = F_86_95[F_86_95['FA11Byggnadstyp']=='Loftgångshus']
# F_86_95_Flerbostadsvilla = F_86_95[F_86_95['FA11Byggnadstyp']=='Flerbostadsvilla']
# F_86_95_AnnanVilken = F_86_95[F_86_95['FA11Byggnadstyp']=='AnnanVilken']
# F_86_95_Radhus = F_86_95[F_86_95['FA11Byggnadstyp']=='Radhus']
# F_96_05_Lamellhus = F_96_05[F_96_05['FA11Byggnadstyp']=='Lamellhus']
# F_96_05_Skivhus = F_96_05[F_96_05['FA11Byggnadstyp']=='Skivhus']
# F_96_05_Punkthus = F_96_05[F_96_05['FA11Byggnadstyp']=='Punkthus']
# F_96_05_Loftgångshus = F_96_05[F_96_05['FA11Byggnadstyp']=='Loftgångshus']
# F_96_05_Flerbostadsvilla = F_96_05[F_96_05['FA11Byggnadstyp']=='Flerbostadsvilla']
# F_96_05_AnnanVilken = F_96_05[F_96_05['FA11Byggnadstyp']=='AnnanVilken']
# F_96_05_Radhus = F_96_05[F_96_05['FA11Byggnadstyp']=='Radhus']

In [ ]:
# U_wall.append(F_60_Lamellhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_Skivhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_Punkthus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_Loftgångshus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_Flerbostadsvilla['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_AnnanVilken['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_60_Radhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Lamellhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Skivhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Punkthus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Loftgångshus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Flerbostadsvilla['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_AnnanVilken['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_61_75_Radhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Lamellhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Skivhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Punkthus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Loftgångshus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Flerbostadsvilla['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_AnnanVilken['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_76_85_Radhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Lamellhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Skivhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Punkthus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Loftgångshus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Flerbostadsvilla['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_AnnanVilken['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_86_95_Radhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Lamellhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Skivhus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Punkthus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Loftgångshus['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Flerbostadsvilla['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_AnnanVilken['FB52 U-medelvärde ytterväggar'].mean())
# U_wall.append(F_96_05_Radhus['FB52 U-medelvärde ytterväggar'].mean())

In [ ]:
# U_win.append(F_60_Lamellhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_Skivhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_Punkthus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_Loftgångshus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_Flerbostadsvilla['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_AnnanVilken['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_60_Radhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Lamellhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Skivhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Punkthus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Loftgångshus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Flerbostadsvilla['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_AnnanVilken['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_61_75_Radhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Lamellhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Skivhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Punkthus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Loftgångshus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Flerbostadsvilla['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_AnnanVilken['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_76_85_Radhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Lamellhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Skivhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Punkthus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Loftgångshus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Flerbostadsvilla['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_AnnanVilken['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_86_95_Radhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Lamellhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Skivhus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Punkthus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Loftgångshus['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Flerbostadsvilla['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_AnnanVilken['FB87-97 U-medelvärde fönster'].mean())
# U_win.append(F_96_05_Radhus['FB87-97 U-medelvärde fönster'].mean())

In [ ]:
# # archetypes seperation
# y_sep = list(F['Year'].unique())
# # building type seperatioin
# bt_sep = list(F['FA11Byggnadstyp'].unique())

# F_in_1 = F.groupby('Classification')
# F_in = F_in_1.agg({'U-medelvärde inkl köldbryggor':'mean',
#                                  'Totalt UA-värde inkl köldbryggor': 'mean', 
#                                  'FB88 Summa fönsterarea N': 'mean', 
#                                  'FB91 Summa fönsterarea S': 'mean', 
#                                  'FB94 Summa fönsterarea V': 'mean', 
#                                  'FB97 Summa fönsterarea O': 'mean', 
#                                  'FB48 Ytterväggsarea N': 'mean',
#                                  'FB48 Ytterväggsarea S': 'mean',
#                                  'FB48 Ytterväggsarea V': 'mean', 
#                                  'FB48 Ytterväggsarea O': 'mean', 
#                                  'F14ATempt': 'mean', 
#                                  'FA20Summa':'mean'})
# F_in

In [ ]:
# (F['FB88 Summa fönsterarea N']/F.loc[:, 'FB48 Ytterväggsarea N']).describe()

In [ ]:
# Umean_incl_cb = F_in['U-medelvärde inkl köldbryggor']
# UAmean_incl_cb = F_in['Totalt UA-värde inkl köldbryggor'] 
# Aenv = UAmean_incl_cb / Umean_incl_cb
# WWR_N = F_in['FB88 Summa fönsterarea N'] / F_in['FB48 Ytterväggsarea N']
# WWR_S = F_in['FB91 Summa fönsterarea S'] / F_in['FB48 Ytterväggsarea S']
# WWR_V = F_in['FB94 Summa fönsterarea V'] / F_in['FB48 Ytterväggsarea V']
# WWR_O = F_in['FB97 Summa fönsterarea O'] / F_in['FB48 Ytterväggsarea O']
# Atemp = F_in['F14ATempt']
# AptNr = F_in['FA20Summa']
# UA_per_Atemp = UAmean_incl_cb / Atemp
# #building aspect ratio N(S):V(o)
# BAR_NS_OV = ((F_in['FB48 Ytterväggsarea N']+F_in['FB48 Ytterväggsarea S'])/2)/((F_in['FB48 Ytterväggsarea O']+F_in['FB48 Ytterväggsarea V'])/2)

In [ ]:
# Arche = pd.DataFrame(columns=['Classification','U-mean incl cb', 'UA-mean incl cb', 'Aenv', 'Atemp', 'UA/Atemp', 'WWR_N', 'WWR_S', 'WWR_V', 'WWR_O', 'BAR', 'Apt No.'])
# Arche['Classification'] = F_in.index
# # Arche['Year'] = F_in['']
# Arche['U-mean incl cb'] = F_in['U-medelvärde inkl köldbryggor'].values
# Arche['UA-mean incl cb'] = F_in['Totalt UA-värde inkl köldbryggor'].values
# Arche['Aenv'] = Aenv.values
# Arche['Atemp'] = Atemp.values
# Arche['UA/Atemp'] = UA_per_Atemp.values
# Arche['WWR_N'] = WWR_N.values
# Arche['WWR_S'] = WWR_S.values
# Arche['WWR_V'] = WWR_V.values
# Arche['WWR_O'] = WWR_O.values
# Arche['BAR'] = BAR_NS_OV.values
# Arche['Apt No.'] = AptNr.values

# Arche

In [ ]:
# Arche.to_excel('Archetypes info.xlsx', sheet_name='Properties')

In [ ]:
# F_in